## Notes
1. Require config.json

    {
	    "api_key" : "xxxxx",
	    "server" : "https://data.reg.cmu.ac.th",
		"csv_file" : "ckan_data.csv"
    }

## 0 Install required packages

In [2]:
import sys
!{sys.executable} -m pip install pandas

You should consider upgrading via the 'C:\Users\pruet\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


## 1 Load libraries and configurations

In [3]:
import json
import pprint
import urllib
import pandas as pd
import requests

# read configuration
config_file = 'config.json'
config = json.load(open(config_file))
api_key = config['api_key']
server = config['server']
csv_file = config['csv_file']


## 2 Function to submit request to server

In [4]:
def send_request_to_server(server, api_key, action_path, dataset_dict):
	headers_dict = {
		'Authorization': api_key,
		'Content-Type' : 'application/x-www-form-urlencoded'
	}
	if dataset_dict != None:
		dataset_dict = urllib.parse.quote(json.dumps(dataset_dict))
	else:
		dataset_dict = []
	req = requests.post(server + 'api/action/' + action_path, headers=headers_dict, data=dataset_dict)
	
	# Use the json module to load CKAN's response into a dictionary.
	return req.json()

## 3 Functions to handling CKAN

In [5]:
def create_package(server, api_key, dataset_dict):
	response = send_request_to_server(server, api_key, 'package_create', dataset_dict)
	if response['success'] == False:
		print(response)
	return response['success']

In [6]:
def update_package(server, api_key, dataset_dict):
	response = send_request_to_server(server, api_key, 'package_update', dataset_dict)
	if response['success'] == False:
		print(response)
	return response['success']

In [7]:
def get_package_info(server, api_key, package_name):
	return send_request_to_server(server, api_key, 'package_show?id=' + package_name, None)

In [8]:
def get_resource_search_name_name(server, api_key, resource_name):
	response = send_request_to_server(server, api_key, 'resource_search?query=name:' + resource_name, None)
	if response['result']['count'] > 0:
		return response['result']['results'][0]
	return None

In [20]:
def create_resource(server, api_key, dataset_dict):
	print(dataset_dict['name'])
	package_info = get_package_info(server, api_key, dataset_dict['name'])
	package_id = package_info['result']['id']
	print(package_id)
	dataset_dict_resource = {
		'package_id' : package_id,
		'url' : dataset_dict['resource_url'],
		'name' : dataset_dict['resource_name'],
		'description' : dataset_dict['resource_description'],
		'format' : dataset_dict['resource_format']
	}
	resource = get_resource_search_name_name(server, api_key, dataset_dict['resource_name'])
	if resource == None:
		response = send_request_to_server(server, api_key, 'resource_create', dataset_dict_resource)
	else:
		dataset_dict_resource['id'] = resource['id']
		response = send_request_to_server(server, api_key, 'resource_update', dataset_dict_resource)
	if response['success'] == False:
		print(response)
	return response['success']

## 4 Main code

In [21]:
df = pd.read_csv(csv_file)
for index, row in df.iterrows():
	tags = row['Tags'].split(',')
	tag_list= []
	for tag in tags:
		tag_list.append({'name' : tag})
	dataset_dict = {
		'owner_org' : row['Organization'],
		'groups' : [
			{'id' : row['Group']}
		],
		'name' : row['Name'],
		'title' : row['Title'],
		'author' : row['Author'],
		'notes' : row['Notes'],
		'author_email' : row['AuthorEmail'],
		'maintainer' : row['Maintainer'],
		'maintainer_email' : row['MaintainerEmail'],
		'url' : row['Url'],
		'tags' : tag_list,
		'resource_name' : row['ResourceName'],
		'resource_url' : row['ResourceUrl'],
		'resource_format' : row['ResourceFormat'],
		'resource_description' : row['ResourceDescripiton'],
		'extras' : [
			{'key' : 'PublicAccess', 'value' : row['PublicAccess']},
			{'key' : 'UniversityAccess', 'value' : row['UniversityAccess']},
			{'key' : 'FacultyAccess', 'value' : row['FacultyAccess']},
			{'key' : 'DepartmentAccess', 'value' : row['DepartmentAccess']},
			{'key' : 'AdvisorAccess', 'value' : row['AdvisorAccess'] },
			{'key' : 'StudentAccess', 'value' : row['StudentAccess']},
			{'key' : 'PDPA', 'value' : row['PDPA']},
			{'key' : 'RetainPolicy', 'value' : row['ReatainPolicy'] },
			{'key' : 'UpdatePolicy', 'value' : row['UpdatePolicy'] }
		]
	}
	#if  update_package(server, api_key, dataset_dict):
	create_resource(server, api_key, dataset_dict)

bulletin
{'help': 'http://10.110.2.208/api/3/action/help_show?name=package_show', 'success': True, 'result': {'author': 'Pruet Boonma', 'author_email': 'pruet.b@cmu.ac.th', 'creator_user_id': '71acce18-1a54-43b2-adb7-23a0494312f6', 'id': 'eedc515e-5ae9-471d-b9e5-3693a51cdfed', 'isopen': False, 'license_id': None, 'license_title': None, 'maintainer': 'Pruet Boonma', 'maintainer_email': 'pruet.b@cmu.ac.th', 'metadata_created': '2021-12-02T02:32:33.408440', 'metadata_modified': '2021-12-02T02:42:20.770734', 'name': 'bulletin', 'notes': "Course information, as of CMU's bulletin.", 'num_resources': 1, 'num_tags': 2, 'organization': {'id': '12b44fd7-a60c-4f0a-a0a0-254f7b307ff3', 'name': 'registrationoffice', 'title': 'Registration Office', 'type': 'organization', 'description': '', 'image_url': 'https://www.reg.cmu.ac.th/webreg/wp-content/uploads/2020/07/cropped-reglogo-1-4.png', 'created': '2021-11-30T16:11:00.017034', 'is_organization': True, 'approval_status': 'approved', 'state': 'active